# 33. Расстояние по Левенштейну

## Информация

легкое

## Условие

Дана текстовая строка. С ней можно выполнять следующие операции:

1. **Заменить** один символ строки на другой символ.
2. **Удалить** один произвольный символ.
3. **Вставить** произвольный символ в произвольное место строки.

Например, при помощи первой операции из строки «СОК» можно получить строку «СУК», при помощи второй операци — строку «ОК», при помощи третьей операции — строку «СТОК».

Минимальное количество таких операций, при помощи которых можно из одной строки получить другую, называется **стоимостью редактирования** или **расстоянием Левенштейна**.

Определите расстояние Левенштейна для двух данных строк.

**Формат ввода**

Программа получает на вход две строки, длина каждой из которых не превосходит 1000 символов, строки состоят только из заглавных латинских букв.

**Формат вывода**

Требуется вывести одно число — расстояние Левенштейна для данных строк.

## Решение №1 Расстояние Левенштейна с использованием рекурсивного подхода.

Пусть у нас будут 2-е строки: `str1` и `str2`, и их длинны соответственно `m`  и  `n`.
Чтобы реализовать рекурсивное решение, нужно расписать базовые и рекурсивные случаи.

> Обход строки идет справа налево!

Базовый случай, если строка у нас в ходе рекурсивного спуска заканчивается, то следует возвращать оставшиеся символы другой строки:

```python
if m == 0:
    return n
if n == 0:
    return m
```

Теперь стоит рассмотреть рекурсивные случаи:

1. Случай, когда символы совпадают, то это не увеличивает расстояние:
   
    ```python
    return levenshteinRecursive(str1, str2, m - 1, n - 1)
    ```

2. Случай, когда символы различаются, то нужно будет найти минимум из 3-x операций - `Insert`, `Delete`, `Replace`:

    **Insert:** Вставляем последний символ `str2[n - 1]` в `str1`.
    Тогда `str2` становится короче `(n-1)`, а `str1` остаётся.    

    **Delete:** Удаляем символ из `str1`, значит она становится короче `(m-1)`.

    **Replace:** Меняем символ `str1[m - 1]` на `str2[n - 1]`, и обе строки сдвигаются назад.

    ```python
    return 1 + min(
        levenshteinRecursive(str1, str2, m, n - 1),  # Insert
        levenshteinRecursive(str1, str2, m - 1, n),  # Remove
        levenshteinRecursive(str1, str2, m - 1, n - 1)  # Replace
    )
    ```

### Готовое решение

In [6]:
def levenshteinRecursive(str1: str, str2: str, m: int, n: int) -> int:
    # base
    if m == 0:
        return n
    if n == 0:
        return m
    
    # recursive
    if str1[m - 1] == str2[n - 1]:
        return levenshteinRecursive(str1, str2, m - 1, n - 1)
    
    return 1 + min(
        levenshteinRecursive(str1, str2, m, n - 1),  # Insert
        levenshteinRecursive(str1, str2, m - 1, n),  # Remove
        levenshteinRecursive(str1, str2, m - 1, n - 1)  # Replace
    )
    
    
s1 = 'ABCDEFGH'
s2 = 'ACDEXGIH'

print(f'Distance = {levenshteinRecursive(s1, s2, len(s1), len(s2))}')

Distance = 3


### Сложность

По времени: $O(3^{m+n})$

По памяти: $O(m+n)$

## Решение №2 Через динамическое программирование.

![matrix](./../imgs/33.jpg)

Идея в том, чтобы создать матрицу, которая показывает, минимальное кол-во операция для преобразования каждой подстроки `str1[:i]` в `str2[:j]`

Инициализация матрицы (пример):

```bash
str1 = "kitten"
str2 = "sitting"
    ''  s  i  t  t  i  n  g
   +------------------------
'' | 0  1  2  3  4  5  6  7
k  | 1
i  | 2
t  | 3
t  | 4
e  | 5
n  | 6
```

- Верхняя строка и первый столбец означают: сколько операций нужно, если одна из строк пуста. Например, чтобы превратить '' в "sitt" нужно 4 вставки.

Будем идти построчно, сравнивая `str1[i - 1]` и `str2[j - 1]`.
Если они равны — просто копируем значение из `dp[i - 1][j - 1]`.
Иначе — берём минимум из трёх возможных операций `+ 1`.

Пример:

```python
dp[1][1] = 1 + min(
    dp[0][1],  # удаление
    dp[1][0],  # вставка
    dp[0][0]   # замена ← минимально
) = 1
```

Значит, чтобы "k" превратить в "s" → одна операция (замена 'k' на 's').

### Готовое решение

In [ ]:
def levenshteinDP(str1: str, str2: str) -> int:
    n = len(str1)  # строки
    m = len(str2)  # столбцы

    dp = [[0 for _ in range(m + 1)] for _ in range(n + 1)]

    # инициализация
    for i in range(n + 1):
        dp[i][0] = i
    for j in range(m + 1):
        dp[0][j] = j

    # заполнение
    for i in range(1, n + 1):
        for j in range(1, m + 1):
            if str1[i - 1] == str2[j - 1]:
                dp[i][j] = dp[i - 1][j - 1]
            else:
                dp[i][j] = 1 + min(
                    dp[i - 1][j],    # удаление
                    dp[i][j - 1],    # вставка
                    dp[i - 1][j - 1] # замена
                )

    return dp[n][m]


s1 = 'ABCDEFGH'
s2 = 'ACDEXGIH'

print(f'Distance = {levenshteinDP(s1, s2)}')

Distance = 3


### Сложность

По времени: $O(n * m)$

По памяти: $O(n * m)$